# Preprocesamiento Corpus

In [1]:
#Librerias para este segmento
import os
from nltk.stem import SnowballStemmer 
import nltk
import string
import re

1. Cargar corpus y stopwords


In [2]:
#Path del corpus
#data_path = r'C:\Users\kevin\OneDrive\Documentos\GitHub\ProyectoRI\data\training_txt'
#data_path = r'D:\U\7. Septimo\RI\ProyectoRI\data\training_txt'
data_path = r'C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\training_txt'

In [3]:
#Path de las stopwords
#stopwords_path = r"C:\Users\kevin\OneDrive\Documentos\GitHub\ProyectoRI\data\stopwords.txt"
#stopwords_path = r"D:\U\7. Septimo\RI\ProyectoRI\data\stopwords.txt"
stopwords_path = r"C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\stopwords.txt"

2. Creamos una tupla con todos los documentos y sus contenidos

In [4]:
documents = [] #  instacia del documents
for filename in os.listdir(data_path):
    if filename.endswith('.txt'): 
        path = os.path.join(data_path, filename) #Abrimos cada archivo 
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read() # leemos
            documents.append((filename, content))

3. Obtenemos las stopwords

In [5]:
with open(stopwords_path, 'r', encoding='utf-8') as file: #Abrimos el archivo 
    stop_words = set(file.read().splitlines()) # leemos las stopwords

In [6]:
#Stemmer se usa para reducir las palabras a su raíz
stemmer = SnowballStemmer('english')

4. Funcion de prepocesamiento

In [7]:
#Definimos una funcion para el preprocesamiento de la data
def preprocess_text(text):
    text = text.lower() #conviertimos en minúsculas
    text = re.sub(r'\d+', '', text)  
    text = text.translate(str.maketrans('', '', string.punctuation))#Eliminamos los signos de puntuación
    tokens = nltk.word_tokenize(text)#tokenizamos
    processed_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words] #aplicamos stemming
    return ' '.join(processed_tokens)

 4. Preprocesamos el contenido de nuestro corpus
 
    La lista de tuplas donde cada tupla contiene:
 
    ["nameFile", "contenido_prepocesado"]

In [8]:
preprocessed_documents = [(filename, preprocess_text(content)) for filename, content in documents]

# Sistema de Recuperacion de informacion

In [9]:
#Librerias para este segmento
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.spatial.distance import jaccard

1. Definimos nuestras Querys ("Consultas")

In [10]:
querys_ = ['acq','alum','barley','bop','carcass','castor-oil','coconut','coconut-oil','cocoa','coffee',
            'copra-cake','copper','corn','cotton','cotton-oil','cpu','cpi','crude','dfl','dmk','dlr','earn',
            'fuel','gas','gnp','gold','grain','groundnut','groundnut-oil','hog','housing','income','instal-debt',
            'interest', 'ipi','iron-steel','jobs','jet','l-cattle','lead','lei','lin-oil','livestock','lumber','meal-feed',
            'money-fx','money-supply','naphtha','nat-gas','nickel','nkr','nzdlr','oat','oilseed','orange','palladium','palm-oil',
            'palmkernel','pet-chem','platinum','potato','propane','rand','rape-oil','rapeseed','reserves','retail','rice',
            'rubber','rye','ship','silver','sorghum','soy-meal','soy-oil','soybean','strategic-metal','sugar','sun-meal',
            'sun-oil','sunseed','tea','tin','trade','veg-oil','wheat','wpi','yen','zinc'
            ]

2. Sistema de recuperacion de la informacion

         1. Extraemos el contenido de las tuplas para vectorizarlas

         2. Vectorizamos el cotenido y la Query

         3. Calculamos la distancia y las convertimos en similitud
         (1 indica una similitud perfecta (los conjuntos son idénticos en términos de elementos no nulos)
         (0 indica ninguna similitud (los conjuntos son completamente diferentes)

In [57]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import jaccard

def sistema_ri(query, texts_prepocesados, umbrales):
    # Método 1: BoW con Jaccard
    # Extraer el contenido de los documentos preprocesados
    contenido_BoW = [doc[1] for doc in texts_prepocesados]  # asumiendo que doc[1] es el contenido preprocesado
    vectorizerBoW = CountVectorizer()
    X_Bow = vectorizerBoW.fit_transform(contenido_BoW)  # vectorizar el contenido de los documentos

    # Vectorizar la consulta
    query_vec_BoW = vectorizerBoW.transform([query]).toarray()[0]

    # Calcular similitudes utilizando la distancia de Jaccard
    similarities_bow = []
    for i in range(X_Bow.shape[0]):
        doc_vec_BoW = X_Bow[i].toarray()[0]
        distance = jaccard(doc_vec_BoW, query_vec_BoW)
        similarity = 1 - distance
        similarities_bow.append(similarity)
    
    

    # Método 2: TF-IDF con similitud del coseno
    contenido_tfidf = [doc[1] for doc in texts_prepocesados]  # Contenido preprocesado
    
    vectorizer_tfidf = TfidfVectorizer()
    X_tfidf = vectorizer_tfidf.fit_transform(contenido_tfidf)
    query_vec_tfidf = vectorizer_tfidf.transform([query])
    
    similarities_tfidf = cosine_similarity(X_tfidf, query_vec_tfidf)
    similarities_tfidf = [sim[0] for sim in similarities_tfidf]

    resultados_bow = {}
    resultados_tfidf = {}
    
    # Iterar sobre los umbrales y obtener resultados para cada uno
    for umbral in umbrales:
        # Filtrar resultados por umbral
        resultados_bow[umbral] = [(texts_prepocesados[idx][0], sim) for idx, sim in enumerate(similarities_bow) if sim >= umbral]
        resultados_tfidf[umbral] = [(texts_prepocesados[idx][0], sim) for idx, sim in enumerate(similarities_tfidf) if sim >= umbral]
    
    return resultados_bow, resultados_tfidf



In [58]:
# Consulta
query = 'earn'

# Obtener resultados de similitud de BoW con Jaccard y TF-IDF con similitud del coseno
#resultados_bow, resultados_tfidf = sistema_ri(query, preprocessed_documents)


In [61]:
# Definir una lista de umbrales
umbrales = [0.03, 0.05, 0.07]

# Obtener resultados para cada umbral
resultados_bow, resultados_tfidf = sistema_ri(query, preprocessed_documents, umbrales)


Resultados para BoW con Jaccard:
Umbral >= 0.03:
Documento: 10025.txt, Similitud Jaccard: 0.0345
Documento: 10090.txt, Similitud Jaccard: 0.0455
Documento: 10096.txt, Similitud Jaccard: 0.0385
Documento: 10114.txt, Similitud Jaccard: 0.0333
Documento: 10181.txt, Similitud Jaccard: 0.0625
Documento: 10185.txt, Similitud Jaccard: 0.0385
Documento: 10440.txt, Similitud Jaccard: 0.0303
Documento: 10841.txt, Similitud Jaccard: 0.0312
Documento: 10973.txt, Similitud Jaccard: 0.0312
Documento: 11443.txt, Similitud Jaccard: 0.0556
Documento: 11683.txt, Similitud Jaccard: 0.0400
Documento: 11722.txt, Similitud Jaccard: 0.0303
Documento: 11895.txt, Similitud Jaccard: 0.0312
Documento: 11904.txt, Similitud Jaccard: 0.0417
Documento: 12085.txt, Similitud Jaccard: 0.0500
Documento: 12202.txt, Similitud Jaccard: 0.0370
Documento: 12346.txt, Similitud Jaccard: 0.0667
Documento: 12551.txt, Similitud Jaccard: 0.0526
Documento: 12831.txt, Similitud Jaccard: 0.0400
Documento: 12860.txt, Similitud Jaccard

# Sistema de recuperacion de informacion COMPLETA

            Todas las Querys con diferentes umbrales

In [64]:
# Definir una lista de umbrales
umbrales = [0.3, 0.5, 0.7]
consultas = querys_
# Función para obtener resultados para todas las consultas y umbrales
def obtener_resultados_para_consultas(consultas, preprocessed_documents, umbrales):
    resultados_por_consulta = {}

    for consulta in consultas:
        resultados_bow, resultados_tfidf = sistema_ri(consulta, preprocessed_documents, umbrales)
        resultados_por_consulta[consulta] = {
            'resultados_bow': resultados_bow,
            'resultados_tfidf': resultados_tfidf
        }

    return resultados_por_consulta

# Obtener resultados para todas las consultas y umbrales
resultados_por_consulta = obtener_resultados_para_consultas(consultas, preprocessed_documents, umbrales)

# Imprimir resultados para cada consulta y umbrales
for consulta in consultas:
    print(f"Resultados para consulta '{consulta}':")
    print("Resultados para BoW con Jaccard:")
    for umbral in umbrales:
        print(f"Umbral >= {umbral}:")
        for doc_id, sim in resultados_por_consulta[consulta]['resultados_bow'][umbral]:
            print(f"Documento: {doc_id}, Similitud Jaccard: {sim:.4f}")

    print()

    print("Resultados para TF-IDF con similitud del coseno:")
    for umbral in umbrales:
        print(f"Umbral >= {umbral}:")
        for doc_id, sim in resultados_por_consulta[consulta]['resultados_tfidf'][umbral]:
            print(f"Documento: {doc_id}, Similitud Coseno: {sim:.4f}")

    print()

Resultados para consulta 'acq':
Resultados para BoW con Jaccard:
Umbral >= 0.3:
Umbral >= 0.5:
Umbral >= 0.7:

Resultados para TF-IDF con similitud del coseno:
Umbral >= 0.3:
Umbral >= 0.5:
Umbral >= 0.7:

Resultados para consulta 'alum':
Resultados para BoW con Jaccard:
Umbral >= 0.3:
Umbral >= 0.5:
Umbral >= 0.7:

Resultados para TF-IDF con similitud del coseno:
Umbral >= 0.3:
Umbral >= 0.5:
Umbral >= 0.7:

Resultados para consulta 'barley':
Resultados para BoW con Jaccard:
Umbral >= 0.3:
Umbral >= 0.5:
Umbral >= 0.7:

Resultados para TF-IDF con similitud del coseno:
Umbral >= 0.3:
Documento: 1067.txt, Similitud Coseno: 0.3588
Documento: 11208.txt, Similitud Coseno: 0.5120
Documento: 11316.txt, Similitud Coseno: 0.3096
Documento: 12428.txt, Similitud Coseno: 0.4247
Documento: 12500.txt, Similitud Coseno: 0.3062
Documento: 13099.txt, Similitud Coseno: 0.3394
Documento: 13744.txt, Similitud Coseno: 0.4871
Documento: 13795.txt, Similitud Coseno: 0.4638
Documento: 1652.txt, Similitud Cos